In [38]:
EMBEDDING_MODEL_NAME = 'deutsche-telekom/gbert-large-paraphrase-cosine'

In [39]:
### SECTION 1.1 Load PDF files

In [40]:
import fitz  # PyMuPDF
from langchain.docstore.document import Document
import docx
import openpyxl
import os

def load_documents(files):
    """
    Loads documents from PDF, DOCX, and XLSX files.

    Parameters:
    - files: A string representing a single file path or a list of strings representing multiple file paths.

    Returns:
    - A list of Document objects loaded from the provided files.

    Raises:
    - FileNotFoundError: If any of the provided file paths do not exist.
    - Exception: For any other issues encountered during file loading.
    """
    if not isinstance(files, list):
        files = [files]  # Ensure 'files' is always a list

    documents = []
    for file_path in files:
        try:
            file_extension = os.path.splitext(file_path)[1].lower()
            if file_extension == '.pdf':
                documents.extend(load_pdf(file_path))
            elif file_extension == '.docx':
                documents.extend(load_docx(file_path))
            elif file_extension == '.xlsx':
                documents.extend(load_xlsx(file_path))
            else:
                print(f"Unsupported file type: {file_extension}")
        except FileNotFoundError as e:
            print(f"File not found: {e.filename}")
            raise
        except Exception as e:
            print(f"An error occurred while loading {file_path}: {e}")
            raise

    return documents

def load_docx(file_path):
    """
    Loads text from a DOCX file.
    """
    doc = docx.Document(file_path)
    text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
    text = clean_extra_whitespace(text)
    text = group_broken_paragraphs(text)
    return [Document(page_content=text, metadata={"source": file_path})]

def load_xlsx(file_path):
    """
    Loads text from an XLSX file.
    """
    wb = openpyxl.load_workbook(file_path)
    text = ""
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        for row in ws.iter_rows(values_only=True):
            text += " ".join([str(cell) for cell in row if cell is not None]) + "\n"
    text = clean_extra_whitespace(text)
    text = group_broken_paragraphs(text)
    return [Document(page_content=text, metadata={"source": file_path})]
    
def load_pdf(file_path):
    """
    Loads documents from a PDF file using PyMuPDF.

    Parameters:
    - file_path: A string representing the path to the PDF file.

    Returns:
    - A list containing a single Document object loaded from the provided PDF file.

    Raises:
    - FileNotFoundError: If the provided file path does not exist.
    - Exception: For any other issues encountered during file loading.

    The function applies post-processing steps such as cleaning extra whitespace and grouping broken paragraphs.
    """
    try:
        # Open the PDF file
        doc = fitz.open(file_path)
        text = ""
        # Extract text from each page
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text += page.get_text("text")

        # Apply post-processing steps
        text = clean_extra_whitespace(text)
        text = group_broken_paragraphs(text)

        # Create a Document object
        document = Document(
            page_content=text,
            metadata={"source": file_path}
        )
        return [document]
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        raise
    except Exception as e:
        print(f"An error occurred while loading {file_path}: {e}")
        raise

def clean_extra_whitespace(text):
    """
    Cleans extra whitespace from the provided text.

    Parameters:
    - text: A string representing the text to be cleaned.

    Returns:
    - A string with extra whitespace removed.
    """
    return ' '.join(text.split())

def group_broken_paragraphs(text):
    """
    Groups broken paragraphs in the provided text.

    Parameters:
    - text: A string representing the text to be processed.

    Returns:
    - A string with broken paragraphs grouped.
    """
    return text.replace("\n", " ").replace("\r", " ")


In [41]:
directory = "C:/Pruebas/RAG Search/demo_docu"
# Get all files in the directory
files = os.listdir(directory)
# Filter out PDF, DOCX, and XLSX files
document_files = [f"{directory}/{file}" for file in files if file.endswith(('.pdf', '.docx', '.xlsx'))]
print(document_files)

['C:/Pruebas/RAG Search/demo_docu/Ansuchen Bildungskarenz.docx', 'C:/Pruebas/RAG Search/demo_docu/Broschuere_Int-Mitarbeitende_2023_WEB.pdf', 'C:/Pruebas/RAG Search/demo_docu/BV_Sonderurlaube_2014-02.pdf', 'C:/Pruebas/RAG Search/demo_docu/BV_Sonderurlaube_Dienstverhinderungen.pdf', 'C:/Pruebas/RAG Search/demo_docu/Doku-An-Abwesenheit-Corona-Krise.xlsx', 'C:/Pruebas/RAG Search/demo_docu/Papamonat_Frühkarenzurlaub_für_Väter.pdf']


In [42]:
documents = load_documents(files=document_files)

In [43]:
len(documents)

6

In [70]:
### SECTION 1.3 Experimenting with Chunk Sizes using RecursiveCharacterTextSplitter

#### Introduction
#We are exploring the effects of various chunk sizes on text segmentation using the RecursiveCharacterTextSplitter from Langchain. This experiment is designed to refine our methods for optimally dividing text.

#### Parameters Explained
#- **Chunk Size**: This parameter sets the length of each text chunk, typically measured in characters. We begin with a predetermined chunk size to monitor how the text is segmented.
#- **Chunk Overlap**: This allows for a slight overlap between consecutive chunks to prevent ideas from being split across two chunks. Initially, the overlap is set to 10% of the chunk size, but adjustments may lead to different results.

#### Purpose
#The objective of this experiment is to investigate how varying chunk size and overlap affect text division. By testing different configurations, we seek to discover a strategy that maintains the coherence of ideas while effectively segmenting the text.


In [45]:
def split_documents(
    chunk_size: int,
    knowledge_base,
    tokenizer_name= EMBEDDING_MODEL_NAME,
):
    """
    Splits documents into chunks of maximum size `chunk_size` tokens, using a specified tokenizer.

    Parameters:
    - chunk_size: The maximum number of tokens for each chunk.
    - knowledge_base: A list of LangchainDocument objects to be split.
    - tokenizer_name: (Optional) The name of the tokenizer to use. Defaults to `EMBEDDING_MODEL_NAME`.

    Returns:
    - A list of LangchainDocument objects, each representing a chunk. Duplicates are removed based on `page_content`.

    Raises:
    - ImportError: If necessary modules for tokenization are not available.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
    )

    docs_processed = (text_splitter.split_documents([doc]) for doc in knowledge_base)
    # Flatten list and remove duplicates more efficiently
    unique_texts = set()
    docs_processed_unique = []
    for doc_chunk in docs_processed:
        for doc in doc_chunk:
            if doc.page_content not in unique_texts:
                unique_texts.add(doc.page_content)
                docs_processed_unique.append(doc)

    return docs_processed_unique

In [46]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer

docs_processed = split_documents(
    512,  # We choose a chunk size adapted to our model
    documents,
    tokenizer_name=EMBEDDING_MODEL_NAME,
)

print(f"Number of chunks: {len(docs_processed)}")

Number of chunks: 88


In [47]:
#### SECTION 1.4 The Embedding Model

In [48]:
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm.autonotebook import tqdm, trange

embedding_model = HuggingFaceEmbeddings(model_name="deutsche-telekom/gbert-large-paraphrase-cosine")

In [49]:
embedding_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='deutsche-telekom/gbert-large-paraphrase-cosine', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [50]:
### Vector Store

In [51]:
# Ruta a la carpeta local con archivos PDF, DOCX y XLSX
#folder_path = "C:/Pruebas/RAG Search/demo_docu_3" #"C:/Users/hernandc/RAG Test/RAG Advanced/data" #"C:/Pruebas/RAG Search/Documentos" #demo_docu_2" #demo_docu" #Documentos"

# Lista para almacenar los datos de todos los documentos
data = []

In [53]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm import tqdm
#from langchain.vectorstores.milvus import Milvus
from langchain_milvus import Milvus
from pymilvus import MilvusClient
import os

# Inicializar el cliente de Milvus
client = MilvusClient()

# Nombre de la colección
collection_name = "uni_test_5_1" #"uni_test" "rag_milvus_webinar"

# Verificar si la colección ya existe
if client.has_collection(collection_name):
    print(f"Cargando la colección existente: {collection_name}")

    # Crear una lista para almacenar los documentos procesados
else:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,  # the maximum number of characters in a chunk: we selected this value arbitrarily
        chunk_overlap=100,  # the number of characters to overlap between chunks
        add_start_index=True,  # If `True`, includes chunk's start index in metadata
        strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
    )
    all_splits = text_splitter.split_documents(docs_processed)

    docs_processed = []
    
    # Iterar sobre los documentos y mostrar el progreso
    for doc in tqdm(all_splits, desc="Procesando documentos"):
        docs_processed.append(doc)

    # Supongamos que Milvus.from_documents permite la inserción en lotes
    batch_size = 1  # Tamaño del lote
    num_batches = len(docs_processed) // batch_size + (1 if len(docs_processed) % batch_size != 0 else 0)

    # Crear el vectorstore con los documentos procesados en lotes
    for i in tqdm(range(num_batches), desc="Insertando documentos en Milvus"):
        batch = docs_processed[i * batch_size:(i + 1) * batch_size]
        Milvus.from_documents(documents=batch, embedding=embedding_model, collection_name=collection_name)

vectorstore = Milvus(collection_name=collection_name, embedding_function=embedding_model)

Insertando documentos en Milvus: 100%|███████████████████████████████████████████████| 168/168 [03:44<00:00,  1.34s/it]


In [54]:
retriever = vectorstore.as_retriever()

In [55]:
#similar_chunks = retriever.get_relevant_documents(query="﻿﻿Mein Vater ist gestorben, wie viel Tage Sonderurlaub bekomme ich?")
similar_chunks = retriever.invoke(input="﻿﻿Mein Vater ist gestorben, wie viel Tage Sonderurlaub bekomme ich?")
similar_chunks

[Document(metadata={'source': 'C:/Pruebas/RAG Search/demo_docu/BV_Sonderurlaube_Dienstverhinderungen.pdf', 'start_index': 0, 'pk': 451633302904136280}, page_content='Abs. 2 Arbeitsruhegesetz [ARG]) für die gemäß ihren religiösen Vorschriften festgelegten Feiertage die unbedingt erforderliche freie Zeit unter Fortzahlung des Entgeltes im Höchstausmaß von zwei Arbeitstagen pro Kalenderjahr zwei Arbeitstagen pro Kalenderjahr zwei Arbeitstagen pro Kalenderjahr zwei Arbeitstagen pro Kalenderjahr. Diese Feiertage sind vom/von der Dienstnehmer/in unverzüglich nach Abschluss des Arbeitsvertrages (bzw bei bestehenden Dienst- /Arbeitsverhältnissen innerhalb eines Monats nach'),
 Document(metadata={'source': 'C:/Pruebas/RAG Search/demo_docu/BV_Sonderurlaube_Dienstverhinderungen.pdf', 'start_index': 0, 'pk': 451633302904136276}, page_content='f. f. f. f. Teilnahme an der Bestattung naher Angehöriger, die nicht im gemeinsamen Haushalt gelebt haben ein Arbeitstag ein Arbeitstag ein Arbeitstag ein Ar

In [56]:
for i, chunks in enumerate(similar_chunks):
    print(f"--------------------------------- chunk # {i} -------------------------------------")
    print(chunks.page_content)

--------------------------------- chunk # 0 -------------------------------------
Abs. 2 Arbeitsruhegesetz [ARG]) für die gemäß ihren religiösen Vorschriften festgelegten Feiertage die unbedingt erforderliche freie Zeit unter Fortzahlung des Entgeltes im Höchstausmaß von zwei Arbeitstagen pro Kalenderjahr zwei Arbeitstagen pro Kalenderjahr zwei Arbeitstagen pro Kalenderjahr zwei Arbeitstagen pro Kalenderjahr. Diese Feiertage sind vom/von der Dienstnehmer/in unverzüglich nach Abschluss des Arbeitsvertrages (bzw bei bestehenden Dienst- /Arbeitsverhältnissen innerhalb eines Monats nach
--------------------------------- chunk # 1 -------------------------------------
f. f. f. f. Teilnahme an der Bestattung naher Angehöriger, die nicht im gemeinsamen Haushalt gelebt haben ein Arbeitstag ein Arbeitstag ein Arbeitstag ein Arbeitstag Seite 5 von 7 g. g. g. g. Teilnahme an der Bestattung der Eltern des Ehepartners/eingetragenen Partners/Lebensgefährten ein Arbeitstag ein Arbeitstag ein Arbeitst

In [57]:
def retrieve_context(query, retriever):
    """
    Retrieves and reranks documents relevant to a given query.

    Parameters:
    - query: The search query as a string.
    - retriever: An instance of a Retriever class used to fetch initial documents.

    Returns:
    - A list of reranked documents deemed relevant to the query.

    """
    retrieved_docs = retriever.get_relevant_documents(query)

    return retrieved_docs

In [58]:
### SECTION 1.5 Putting Everything Together

In [71]:
### Using AZURE OPENAI API

#Moving forward, we will be using both openai LLM and Embedding model.

In [60]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(
        (
            "Bitte beantworte die folgende Anfrage auf der Grundlage des angegebenen `Kontext`, der auf die Anfrage folgt.\n"
            "Wenn du die Antwort nicht weißt, dann sag einfach 'Ich weiß es nicht'.\n"
            "Anfrage: {question}\n"
            "Kontext: ```{context}```\n"
        )
    )

In [61]:
from dotenv import load_dotenv
import os
from pathlib import Path

dotenv_path = Path(r'C:\Users\hernandc\RAG Test\apikeys.env')
load_dotenv(dotenv_path=dotenv_path)

True

In [62]:
from openai import AzureOpenAI

# Configurar el cliente de Azure OpenAI
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2023-05-15",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

MODEL = "gpt-4-turbo" # "gpt-4"  # Reemplaza esto con el nombre de tu despliegue de GPT-4 en Azure

In [63]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import HumanMessage

def azure_openai_call(prompt):
    # Si el prompt es un objeto HumanMessage, extraemos su contenido
    if isinstance(prompt, HumanMessage):
        prompt_content = prompt.content
    else:
        prompt_content = str(prompt)
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt_content}
        ]
    )
    return response.choices[0].message.content

llm = (lambda x: azure_openai_call(x))  # Envolver la llamada en una función lambda
chain = prompt_template | llm | StrOutputParser()

In [64]:
query = "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"

context = retrieve_context(
        query, retriever=retriever,
    )

In [65]:
context

[Document(metadata={'source': 'C:/Pruebas/RAG Search/demo_docu/Broschuere_Int-Mitarbeitende_2023_WEB.pdf', 'start_index': 0, 'pk': 451633302904136032}, page_content='um Ihr Arbeitsverhältnis 12 Häufig gestellte Fragen 14 Services & Infos im Überblick Ein Überblick für den Start 16 Über die Uni Graz © Uni Graz/Konstantinov 5 Über die Universität Die Universität Graz, gegründet 1585, ist Österreichs zweitälteste Universität und eine der größten des Landes. Zahlreiche herausragende Wissenschafter:innen, unter ihnen sechs Nobelpreisträger, haben hier gelehrt und geforscht. Mit rund 30.000 Studierenden und 4.500 Mitarbeitenden trägt sie entscheidend zum pulsierenden Leben'),
 Document(metadata={'source': 'C:/Pruebas/RAG Search/demo_docu/Broschuere_Int-Mitarbeitende_2023_WEB.pdf', 'start_index': 417, 'pk': 451633302904136054}, page_content='am Institut für Öffentliches Recht und Politikwissenschaft und leitete bis September 2022 die Abteilung Finanzen, Personal und Recht der Universität für 

In [66]:
text = ""
for ch in context:
    text += ch.page_content

In [67]:
print(text)

um Ihr Arbeitsverhältnis 12 Häufig gestellte Fragen 14 Services & Infos im Überblick Ein Überblick für den Start 16 Über die Uni Graz © Uni Graz/Konstantinov 5 Über die Universität Die Universität Graz, gegründet 1585, ist Österreichs zweitälteste Universität und eine der größten des Landes. Zahlreiche herausragende Wissenschafter:innen, unter ihnen sechs Nobelpreisträger, haben hier gelehrt und geforscht. Mit rund 30.000 Studierenden und 4.500 Mitarbeitenden trägt sie entscheidend zum pulsierenden Lebenam Institut für Öffentliches Recht und Politikwissenschaft und leitete bis September 2022 die Abteilung Finanzen, Personal und Recht der Universität für Weiterbildung Krems. © Uni Graz/WildundWunderbar © Uni Graz/WildundWunderbar Organigramm FAKULTÄTEN* UNIRAT Der Unirat besteht aus 9Graz. Seine beruflichen Stationen führten ihn unter anderem ins Europäische Parlament in Brüssel, in die Steiermärki­ sche Landesregierung, 2002 ins Bundeskanzleramt, mit den Schwerpunkten Wirtschaft und Fo

In [68]:
response = chain.invoke({"context": context, "question": query})

In [69]:
print(response)

Der Rektor der Universität Graz ist seit Oktober 2022 Peter Riedler. Das Geburtsjahr wird im bereitgestellten Kontext nicht angegeben.
